In [ ]:
from google.cloud import bigquery
from dataclasses import dataclass
from datetime import date, timedelta
from dotenv import load_dotenv
import os
from typing import List

load_dotenv()

## Fetch GCP data

In [ ]:
project = os.environ["GCP_PROJECT_ID"]
client = bigquery.Client(project=project)

In [ ]:
@dataclass
class Site:
    ga4_property_id: str
    
DALLAS_FREE_PRESS = Site(ga4_property_id=os.environ["DALLAS_FREE_PRESS_GA4_PROPERTY_ID"])

In [ ]:
def enumerate_query_dates(end_date: date, num_days: int) -> List[date]:
    return [end_date - timedelta(days=i) for i in range(0, num_days)]

def construct_event_table_name(gcp_project_id: str, site: Site, dt: date) -> str:
    return f"{gcp_project_id}.analytics_{site.ga4_property_id}.event_{dt.strftime('%Y%m%d')}"

num_days = 1
query_dates = enumerate_query_dates(date(2023, 10, 15), num_days)

In [ ]:
construct_event_table_name(project, DALLAS_FREE_PRESS, dt=query_dates[0])